# Fiddler Strands Agent Integration Quick Start

## Goal

This guide demonstrates how to integrate **Strands agents** with **Fiddler** for comprehensive observability and monitoring using OpenTelemetry (OTEL). You will learn to instrument agents, configure trace export, and verify monitoring in Fiddler. 🤖

## About Strands Agents

Strands is a modern AI agent framework that provides built-in telemetry support through OpenTelemetry. This notebook shows you how to connect Strands agents to Fiddler's monitoring platform, enabling you to:

- Track agent execution traces and spans
- Monitor tool usage and performance
- Analyze multi-agent workflows
- Debug agent behavior with detailed telemetry
- Set up alerts for agent failures and anomalies

## About Fiddler

Fiddler is the all-in-one AI Observability and Security platform for responsible AI. Monitoring and analytics capabilities provide a common language, centralized controls, and actionable insights to operationalize production ML models, GenAI, AI agents, and LLM applications with trust. Fortune 500 organizations utilize Fiddler to scale LLM and ML deployments, delivering high-performance AI, reducing costs, and ensuring responsible governance.

## Integration Overview

The Fiddler-Strands integration uses three main components:

1. **FiddlerSpanProcessor**: Custom span processor for attribute denormalization
2. **StrandsTelemetry**: Telemetry setup that configures OpenTelemetry
3. **Agent Configuration**: Proper instrumentation of Strands agents

---

## Notebook Summary

In this notebook, we will:

1. Set up environment variables for Fiddler OTLP endpoint
2. Implement a custom FiddlerSpanProcessor for attribute propagation
3. Configure Strands telemetry with OTLP export
4. Create and instrument a Strands agent with tools
5. Execute the agent and send traces to Fiddler
6. Verify traces appear in the Fiddler UI

This notebook should complete within 2-3 minutes.

---

## Prerequisites

- Fiddler account with Application UUID and Bearer Token
- OpenAI API key
- Python packages: `strands-agents`, `opentelemetry-api`, `opentelemetry-sdk`, `opentelemetry-exporter-otlp`

## 1. Install Required Packages

First, let's install all necessary dependencies.

In [ ]:
# Install required packages
%pip install -q strands-agents opentelemetry-api opentelemetry-sdk opentelemetry-exporter-otlp openai

## 2. Configure Environment Variables

Set up your Fiddler credentials and OpenAI API key. Replace the placeholder values with your actual credentials.

**Where to find these values:**
- **FIDDLER_ENDPOINT**: Your Fiddler instance URL (e.g., `http://demo.fiddler.ai`)
- **APPLICATION_UUID**: From your Fiddler application settings
- **BEARER_TOKEN**: Generate from Fiddler Settings → API Tokens
- **OPENAI_API_KEY**: Your OpenAI API key

In [ ]:
import os

# Fiddler Configuration
FIDDLER_ENDPOINT = ''  # e.g., "http://demo.fiddler.ai"
APPLICATION_UUID = ''  # Your Fiddler application UUID
TOKEN = ''  # Your Fiddler access token

# OpenAI Configuration
OPENAI_API_KEY = ''  # Your OpenAI API key

# Set environment variables
os.environ['OTEL_EXPORTER_OTLP_ENDPOINT'] = FIDDLER_ENDPOINT
os.environ['OTEL_RESOURCE_ATTRIBUTES'] = f'application.id={APPLICATION_UUID}'
os.environ['OTEL_EXPORTER_OTLP_HEADERS'] = (
    f'authorization=Bearer {TOKEN},fiddler-application-id={APPLICATION_UUID}'
)
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

print('✅ Environment variables configured successfully')

## 3. Implement FiddlerSpanProcessor

The FiddlerSpanProcessor ensures that critical agent attributes (name, ID, system prompt) are propagated from parent spans to child spans. This is essential for maintaining context in multi-step agent workflows.

In [ ]:
from opentelemetry.sdk.trace import SpanProcessor
from opentelemetry.trace import Span, get_current_span
from opentelemetry import context


class FiddlerSpanProcessor(SpanProcessor):
    """Custom span processor for Fiddler integration.

    Copies critical attributes from parent spans to child spans to ensure
    proper context propagation in agent workflows.
    """

    # Attributes to denormalize (copy from parent to child)
    DENORMALIZED_ATTRIBUTES = ['gen_ai.agent.name', 'gen_ai.agent.id', 'system_prompt']

    def on_start(self, span: Span, parent_context: context.Context = None):
        """Called when a span starts.

        Args:
            span: The span that is starting
            parent_context: Context of the parent span
        """
        # Get the parent span from the context
        parent_span = get_current_span(parent_context)

        # Check if parent span is valid and has attributes
        if (
            parent_span
            and parent_span.is_recording()
            and hasattr(parent_span, 'attributes')
        ):
            # Get all attributes from the parent span
            parent_attributes = parent_span.attributes

            # Copy denormalized attributes to the child span
            for attr in self.DENORMALIZED_ATTRIBUTES:
                if parent_attributes.get(attr):
                    span.set_attribute(attr, parent_attributes.get(attr))


print('✅ FiddlerSpanProcessor implemented')

## 4. Configure Strands Telemetry

Set up the Strands telemetry system with:
- Console exporter for local debugging
- OTLP exporter for sending traces to Fiddler
- FiddlerSpanProcessor for attribute propagation

In [ ]:
from strands.telemetry import StrandsTelemetry

# Initialize Strands telemetry
strands_telemetry = StrandsTelemetry()

# Add the Fiddler span processor for attribute denormalization
strands_telemetry.tracer_provider.add_span_processor(FiddlerSpanProcessor())

# Setup exporters
strands_telemetry.setup_console_exporter()  # For debugging - prints to console
strands_telemetry.setup_otlp_exporter()  # For Fiddler - sends traces via OTLP

print('✅ Strands telemetry configured with Fiddler integration')

## 5. Define Agent Tools

Create tools that the agent can use. These will be automatically traced and monitored by Fiddler.

In [ ]:
from strands import tool

@tool
def get_weather(city: str) -> str:
    """Get the current weather for a city.

    Args:
        city: The name of the city

    Returns:
        Weather information as a string
    """
    # Simulate weather API response
    import random

    conditions = ['sunny', 'cloudy', 'rainy', 'partly cloudy']
    temp = random.randint(60, 85)
    return f'The weather in {city} is {random.choice(conditions)} with a temperature of {temp}°F'

@tool
def search_knowledge_base(query: str) -> str:
    """Search the knowledge base for information.

    Args:
        query: The search query

    Returns:
        Relevant information from the knowledge base
    """
    # Simulate knowledge base search
    knowledge = {
        'monitoring': 'Fiddler provides comprehensive AI observability for models, LLMs, and agents.',
        'agents': 'AI agents can be monitored using OpenTelemetry integration with Fiddler.',
        'strands': 'Strands is a modern agent framework with built-in telemetry support.',
    }

    for key, value in knowledge.items():
        if key in query.lower():
            return value

    return f"No specific information found for '{query}', but Fiddler can help monitor AI systems."


@tool
def calculate(expression: str) -> str:
    """Safely evaluate a mathematical expression.

    Args:
        expression: Mathematical expression to evaluate (e.g., "2 + 2")

    Returns:
        The result of the calculation
    """
    try:
        # Safe evaluation of simple math expressions
        result = eval(expression, {'__builtins__': {}}, {})
        return f'The result is: {result}'
    except Exception as e:
        return f"Error calculating '{expression}': {str(e)}"


print('✅ Agent tools defined: get_weather, search_knowledge_base, calculate')

## 6. Create and Configure the Agent

Initialize the Strands agent with OpenAI model and the defined tools. The agent is automatically instrumented with telemetry.

In [ ]:
from strands import Agent
from strands.models.openai import OpenAIModel

# Initialize OpenAI model
model = OpenAIModel(
    client_args={'api_key': os.getenv('OPENAI_API_KEY')}, model_id='gpt-4o-mini'
)

# Create the agent with system prompt and tools
agent = Agent(
    model=model,
    system_prompt="""You are a helpful AI assistant with access to weather information, 
    a knowledge base, and calculation tools. Always be concise and accurate in your responses.
    When appropriate, use the available tools to provide helpful information.""",
    tools=[get_weather, search_knowledge_base, calculate],
)

print('✅ Agent created and instrumented with telemetry')
print(f'   Model: {model.config}')
print(f'   Tools: {agent.tool_names} available')

## 7. Execute Agent Tasks

Run several example queries to generate traces. Each execution will create spans that are sent to Fiddler.

In [ ]:
import time

# Example 1: Weather query (uses get_weather tool)
print('\n' + '=' * 60)
print('Query 1: Weather Information')
print('=' * 60)
response1 = agent("What's the weather like in San Francisco?")
print(f'Response: {response1}')

time.sleep(1)  # Brief pause between queries

# Example 2: Knowledge base query (uses search_knowledge_base tool)
print('\n' + '=' * 60)
print('Query 2: Knowledge Base Search')
print('=' * 60)
response2 = agent('Tell me about monitoring AI agents with Fiddler')
print(f'Response: {response2}')

time.sleep(1)

# Example 3: Calculation (uses calculate tool)
print('\n' + '=' * 60)
print('Query 3: Mathematical Calculation')
print('=' * 60)
response3 = agent('What is 15 * 7 + 23?')
print(f'Response: {response3}')

time.sleep(1)

# Example 4: Multi-tool query (may use multiple tools)
print('\n' + '=' * 60)
print('Query 4: Complex Multi-Tool Request')
print('=' * 60)
response4 = agent(
    "What's the weather in New York and how does Strands agent monitoring work?"
)
print(f'Response: {response4}')

print('\n' + '=' * 60)
print('✅ All queries completed successfully')
print('=' * 60)

## 8. Verify Traces in Fiddler

After running the agent queries, traces should appear in your Fiddler application within 30 seconds.

**Steps to verify:**

1. Navigate to your Fiddler instance
2. Go to your application (the one with APPLICATION_UUID)
3. Click on the **Traces** tab
4. Look for recent traces from the agent executions

**What to check:**
- ✅ Traces appear for each agent query
- ✅ Parent and child spans are properly linked
- ✅ Tool calls appear as separate spans
- ✅ Agent attributes are present:
  - `gen_ai.agent.name`
  - `gen_ai.agent.id`
  - `system_prompt`
- ✅ Latency metrics are captured

**Console Output:**

You should also see trace information in the console output above (from the console exporter). This helps with local debugging before checking Fiddler.

## 9. Troubleshooting

If traces are not appearing in Fiddler, check the following:

In [ ]:
# Diagnostic checks
print('Diagnostic Information:')
print('=' * 60)

# Check environment variables
print('\n1. Environment Variables:')
print(f'   OTEL_EXPORTER_OTLP_ENDPOINT: {os.getenv("OTEL_EXPORTER_OTLP_ENDPOINT")}')
print(f'   OTEL_RESOURCE_ATTRIBUTES: {os.getenv("OTEL_RESOURCE_ATTRIBUTES")}')
print(
    f'   OTEL_EXPORTER_OTLP_HEADERS: {"Set" if os.getenv("OTEL_EXPORTER_OTLP_HEADERS") else "Not Set"}'
)

# Check network connectivity
print('\n2. Network Connectivity:')
try:
    import requests

    response = requests.head(FIDDLER_ENDPOINT, timeout=5)
    print(
        f'   Connection to {FIDDLER_ENDPOINT}: ✅ Success (Status: {response.status_code})'
    )
except Exception as e:
    print(f'   Connection to {FIDDLER_ENDPOINT}: ❌ Failed ({str(e)})')

# Verify configuration
print('\n3. Configuration:')
print(f'   Fiddler Endpoint: {FIDDLER_ENDPOINT}')
print(f'   Application UUID: {APPLICATION_UUID}')
print(f'   Bearer Token: {"Set" if TOKEN else "Not Set"}')
print(f'   OpenAI API Key: {"Set" if OPENAI_API_KEY else "Not Set"}')

print('\n' + '=' * 60)
print('\nIf issues persist:')
print('1. Verify your personal access token is valid and not expired')
print('2. Ensure Application UUID matches your Fiddler application')
print('3. Check Fiddler application has OTEL ingestion enabled')
print('4. Review console exporter output for trace details')

## 10. Advanced Configuration (Optional)

For production deployments, you may want to customize the telemetry configuration:

In [ ]:
# Example: Advanced telemetry configuration
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import Resource

# Create custom resource with additional metadata
resource = Resource.create(
    {
        'service.name': 'strands-agent-service',
        'service.version': '1.0.0',
        'deployment.environment': 'notebook-demo',
        'application.id': APPLICATION_UUID,
    }
)

# Initialize tracer provider with custom resource
provider = TracerProvider(resource=resource)

# Add Fiddler span processor
provider.add_span_processor(FiddlerSpanProcessor())

# Configure OTLP exporter with custom settings
otlp_exporter = OTLPSpanExporter(
    endpoint=FIDDLER_ENDPOINT,
    headers={
        'authorization': f'Bearer {TOKEN}',
        'fiddler-application-id': APPLICATION_UUID,
    },
    timeout=10,  # Custom timeout in seconds
)

# Add batch processor with custom settings
batch_processor = BatchSpanProcessor(
    otlp_exporter,
    max_queue_size=2048,  # Max spans in queue
    max_export_batch_size=512,  # Spans per export batch
    schedule_delay_millis=5000,  # Export every 5 seconds
)
provider.add_span_processor(batch_processor)

print('✅ Advanced telemetry configuration ready')
print('   This configuration can be used for production deployments')

## Next Steps

Now that you have Strands agents integrated with Fiddler:

### Monitoring & Analytics
- Explore agent performance metrics in Fiddler
- Set up alerts for agent failures and anomalies
- Analyze tool usage patterns
- Track latency and token usage

### Advanced Features
- Implement multi-agent workflows with monitoring
- Add custom metrics and attributes
- Configure sampling strategies for scale
- Set up data privacy and PII filtering

### Resources
- [Fiddler Documentation](https://docs.fiddler.ai)
- [Strands Agents Documentation](https://strands-ai.github.io/strands-agents/)
- [OpenTelemetry Python](https://opentelemetry.io/docs/instrumentation/python/)

---

**Need Help?**
- Email: support@fiddler.ai